# Pytorch网络模型存储与调用以及修改
pytorh网络模型的存储和调用主要利用了`torch.save()`和`torch.load()`这两个函数<br>
pytorch搭建神经网络都是按照父子网络关系生成的,通过层层调用网络的名称可以从中取出部分网络,如果继承了`nn.Module`创建的网络,需要自己手写`forward()`函数来指定网络的的层次递进关系,因为只是`__init__`定义了很多网络模块,网络之间的关系需要手工定义

In [1]:
import torch
import torch.nn as nn

高级一点的网络模型保存和调用可以用如下形式的函数

In [ ]:
def save_checkpoint(model, epoch, save_mode='generator'):
    model_out_path = save_mode+'model/'+'model_epoch_{}.pkl'.format(epoch)
    state = {'epoch':epoch, 'model':model}#使用字典的方式进行存储，可以将epoch编号和网络参数都存储下来
    if not os.path.exists('model/'):#判断当前路径下文件夹是否存在，注意`model`后面要有`/`表示文件夹
        os.makedirs('model/')
    torch.save(state,model_out_path)
    print('Checkpoint saved to {}'.format(model_out_path))#使用format格式化输出

In [ ]:
def load_checkpoint(epoch, model, model_path):
    if os.path.isfile(model_path):
        print('====>loading checkpoint:{}'.format(model_path))
        checkpoint = torch.load(model_path)#注意要先用load加载出网络
        start_epoch += 1
        model.load_state_dict(checkpoint['model'].state_dict())#读取网络模型参数，注意checkpoint是用字典存储的，索引model才是之前训练的网络模型
    print('There is no file named :{}'.format(model_path))

`torch.save(obj, f, pickle_module)`<br>
**功能：**存储一个对象至本地文件,相当于把计算机内存里的对象存到本地，任意obj的类型都可以存储，load后可以继续使用。<br>
**参数：**<br>
-obj:待存储的对象<br>
-f:存储的文件名，string类型

`torch.load(f, map_location=None, pickle_module)`<br>
**功能：**读取save存储的文件，返回save相应的obj<br>
**参数：**<br>
f:文件名称

`torch.nn.Module.load_state_dict(dict)`
**功能：**网络模型载入已有的参数字典<br>
**参数：**
state_dict:参数字典

`torch.nn.Module.state_dict()`<br>
**功能：**返回包括网络所有参数的一个字典<br>
**参数：**可以像字典一样使用,调用`.keys()`可以查看里面的键值

In [18]:
x = torch.ones(2,2,requires_grad=True)
y = x + 2
print('y', y)
z = y*y*3#tensor的乘法是对应位置相乘，不是矩阵相乘
print('z:', z)
out = torch.sum(z)
print('out', out.requires_grad)
out.backward()
print('x', )

y tensor([[ 3.,  3.],
        [ 3.,  3.]])
z: tensor([[ 27.,  27.],
        [ 27.,  27.]])
out True
x


`nn.Sequential(*args)`里面使用的参数收集,将收集到的参数汇成一个序列,Seq生成的网络结构支持索引,即内部网络是按索引来取的,不是按照名称!!!这也体现了pytorch多样化的网络搭建方式  
`nn.ModuleList(list)`将list中的模块搭建成一个网络,注意与Seq的区别,传入的是list  
>这两种方式搭建的网络都是以list的感觉存在(index:对应的网络实例),传统的搭网络是以字典的方式构建(网络名称:对应的网络实例)
这种方式有个好处可以搭建具有很多重复模块的大网络,其中很多子模块不需要给一些专有的名称,只需要使用index编号即可



In [6]:
block = nn.Sequential(*[nn.Conv2d(1,1,3) for i in range(5)])#里面是参数收集,不是seq收集
models = nn.ModuleList([block for i in range(10)])#初始化使用list创建
models[0]#支持index索引
# for i in models:#支持迭代
#     print(i)
models[0][0]#

Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1))

In [5]:
models.append(nn.Conv2d(1,1,1))#使用append直接添加
models[0:2]#支持切片索引等骚操作

NameError: name 'models' is not defined

In [12]:
l = models[0:2]

如果想指定每一个网络名称的具体名称,可以通过`add_module()`方式来完成,`nn.Sequential()和nn.ModuleList()`都支持这个操作,要注意层次关系,`add`进去的module是在当前list的层级,如果里面子块

In [2]:
block = nn.Sequential()#先使用Sequential创建一个空的Seq
block.add_module('conv', nn.Conv2d(1,1,3))#
block.add_module('bn', nn.BatchNorm2d(1))
models = nn.ModuleList()
models.add_module('block1', block)
models.add_module('block2', block)
models.append(block)#使用append默认使用index标号进行命名
models

ModuleList(
  (block1): Sequential(
    (conv): Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1))
    (bn): BatchNorm2d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (block2): Sequential(
    (conv): Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1))
    (bn): BatchNorm2d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (2): Sequential(
    (conv): Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1))
    (bn): BatchNorm2d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
)

In [5]:
models.parameters

<bound method Module.parameters of ModuleList(
  (block1): Sequential(
    (conv): Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1))
    (bn): BatchNorm2d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (block2): Sequential(
    (conv): Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1))
    (bn): BatchNorm2d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (2): Sequential(
    (conv): Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1))
    (bn): BatchNorm2d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
)>

In [18]:
nn.DataParallel(models)

DataParallel(
  (module): ModuleList(
    (block1): Sequential(
      (conv): Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1))
      (bn): BatchNorm2d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (block2): Sequential(
      (conv): Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1))
      (bn): BatchNorm2d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (2): Sequential(
      (conv): Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1))
      (bn): BatchNorm2d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
)

In [27]:
import torch.nn.functional as F

In [36]:
a = torch.tensor([1,2,3,4])
F.dropout2d(a,p=0.5)

tensor([ 1,  2,  3,  4])

In [25]:
drop_layer = nn.Dropout2d(0.5)
for i in drop_layer.parameters():
    print(i.shape)

In [7]:
for name, para in models.state_dict().items():
    print(name, para.type())#结构是

block1.conv.weight torch.FloatTensor
block1.conv.bias torch.FloatTensor
block1.bn.weight torch.FloatTensor
block1.bn.bias torch.FloatTensor
block1.bn.running_mean torch.FloatTensor
block1.bn.running_var torch.FloatTensor
block1.bn.num_batches_tracked torch.LongTensor
block2.conv.weight torch.FloatTensor
block2.conv.bias torch.FloatTensor
block2.bn.weight torch.FloatTensor
block2.bn.bias torch.FloatTensor
block2.bn.running_mean torch.FloatTensor
block2.bn.running_var torch.FloatTensor
block2.bn.num_batches_tracked torch.LongTensor
2.conv.weight torch.FloatTensor
2.conv.bias torch.FloatTensor
2.bn.weight torch.FloatTensor
2.bn.bias torch.FloatTensor
2.bn.running_mean torch.FloatTensor
2.bn.running_var torch.FloatTensor
2.bn.num_batches_tracked torch.LongTensor


# 网络参数的调用和修改
在fine-tuning的时候或者对网络参数进行修改的时候，需要对网络tensor进行调整，因此需要学会对生成的网络对象进行参数读取<br>
调用网络参数的方式有多种可以使用，但是目的只有一个，就是要访问到`tensor`级的参数，以方便我们对网络参数的修改<br>

* `model.state_dict()`返回一个网络的字典,包含名称(str)和参数(nn.Parameter)
* `model.parameters()`返回全部的网络参数的迭代器,没有名称
* `model.modules()`返回网络全部的子模块,这个返回的是一个生成器,只有配合迭代器才有用(for循环)

In [8]:
class TestNet(nn.Module):
    def __init__(self):
        super(TestNet, self).__init__()
        self.conv1 = SubNet1()
        self.conv2 = SubNet2()
        self.l1 = nn.Linear(10, 10)
        self.l2 = nn.Linear(10, 15)
        self.l3 = nn.Linear(15, 1)
        self.test = nn.Linear(15,15)
    def forward(self, x):
        x = self.conv1(x)
        x = self.l1(x)
        x = self.l2(x)
        out = nn.Sigmoid(self.l3(x))
        return out
    def set_trainable(self, layer_regex, model=None, indent=0, verbose=1):
        for param in self.named_parameters():
            layer_name = param[0]
            trainable = bool(re.fullmatch(layer_regex, layer_name))
            if not trainable:
                param[1].requires_grad = False            
class SubNet1(nn.Module):
    def __init__(self):
        super(SubNet1, self).__init__()
        self.conv1 = nn.Conv2d(2, 10, 1)
    def forward(self, x):
        x = self.conv1(x)
        return x        
class SubNet2(nn.Module):
    def __init__(self):
        super(SubNet2, self).__init__()
        self.conv2 = nn.Conv2d(1, 1, 1)
    def forward(self, x):
        x = self.conv2(x)
        return x

In [9]:
test = TestNet()
test.conv1#直接调用网络的属性名称可以取出部分网络

SubNet1(
  (conv1): Conv2d(2, 10, kernel_size=(1, 1), stride=(1, 1))
)

In [10]:
for p in test.parameters():
    print(p.shape)#conv2的bias的shape与out_channel一致，即每一组使用一个偏置

torch.Size([10, 2, 1, 1])
torch.Size([10])
torch.Size([1, 1, 1, 1])
torch.Size([1])
torch.Size([10, 10])
torch.Size([10])
torch.Size([15, 10])
torch.Size([15])
torch.Size([1, 15])
torch.Size([1])
torch.Size([15, 15])
torch.Size([15])


In [26]:
for i in models.named_modules():#这是一个生成器
    print(i)
# models[11]

('', ModuleList(
  (0): Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1))
  (1): Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1))
  (2): Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1))
  (3): Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1))
  (4): Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1))
  (5): Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1))
  (6): Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1))
  (7): Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1))
  (8): Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1))
  (9): Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1))
  (10): Conv2d(1, 1, kernel_size=(1, 1), stride=(1, 1))
))
('0', Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1)))
('1', Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1)))
('2', Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1)))
('3', Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1)))
('4', Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1)))
('5', Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1)))
('6', Conv2d(1, 1, kernel_size=(3, 3), strid

In [ ]:
def prnt_weight(m):
    print(m.weight)
def init_weight(m):
    print(m)
    if type(m) == nn.Linear:
        m.weight.data.fill_(5.0)
net = nn.Sequential(nn.Linear(2,2), nn.Linear(2,2))
net.apply(init_weight)
net.apply(prnt_weight)

class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.conv1 = nn.Conv2d(4,10,kernel_size=3)
        self.fc = nn.Linear(10, 3)

model = Net()
for name, parameter in model.state_dict().items():
    print(name, parameter.data.shape)#可以看到网络的参数shape是(input_channels*output_channels*kernel*kernel)

In [36]:
bn = 'sub.bn'
not 'bn' in bn

False

In [39]:
trainable_en = [para for i,para in net.named_parameters() if para.requires_grad and not 'bn' in name]
trainable_en

[Parameter containing:
 tensor([[[[-0.2338]]],
 
 
         [[[ 0.5962]]],
 
 
         [[[-0.7837]]],
 
 
         [[[ 0.7938]]],
 
 
         [[[-0.4047]]],
 
 
         [[[ 0.9081]]],
 
 
         [[[-0.5584]]],
 
 
         [[[ 0.1025]]],
 
 
         [[[ 0.9495]]],
 
 
         [[[-0.5842]]]]), Parameter containing:
 tensor([ 0.0888, -0.4299,  0.8976, -0.8497,  0.3468, -0.0633, -0.6725,
         -0.6541,  0.1189, -0.2832]), Parameter containing:
 tensor([[[[ 0.8568]]]]), Parameter containing:
 tensor([-0.4776]), Parameter containing:
 tensor([[-0.3057,  0.1490, -0.0670, -0.2937,  0.2081,  0.0296,  0.1956,
           0.1292, -0.2050, -0.0941],
         [ 0.1719,  0.2971, -0.1711,  0.0143, -0.2868,  0.1422, -0.0949,
          -0.1421,  0.1687, -0.0376],
         [-0.0662,  0.0373,  0.2102, -0.1170,  0.1763, -0.2743, -0.1340,
           0.1100,  0.1372,  0.1612],
         [ 0.1908,  0.0613,  0.1323,  0.0355,  0.1914, -0.2417, -0.0268,
           0.1024,  0.0134, -0.0895],
         [ 

In [34]:
layer_regex = {
    # all layers but the backbone
    "heads": r"(fpn.P5\_.*)|(fpn.P4\_.*)|(fpn.P3\_.*)|(fpn.P2\_.*)|(rpn.*)|(classifier.*)|(mask.*)",
    # From a specific Resnet stage and up
    "3+": r"(fpn.C3.*)|(fpn.C4.*)|(fpn.C5.*)|(fpn.P5\_.*)|(fpn.P4\_.*)|(fpn.P3\_.*)|(fpn.P2\_.*)|(rpn.*)|(classifier.*)|(mask.*)",
    "4+": r"(fpn.C4.*)|(fpn.C5.*)|(fpn.P5\_.*)|(fpn.P4\_.*)|(fpn.P3\_.*)|(fpn.P2\_.*)|(rpn.*)|(classifier.*)|(mask.*)",
    "5+": r"(fpn.C5.*)|(fpn.P5\_.*)|(fpn.P4\_.*)|(fpn.P3\_.*)|(fpn.P2\_.*)|(rpn.*)|(classifier.*)|(mask.*)",
    # All layers
    "all": ".*",
}
net = TestNet()
for name, para in net.named_parameters():#named_parameters()返回名称和具体的参数
    print('current name:', name,bool(re.fullmatch(r'.* ', name)))#那么属于匹配项中的规则,返回除

current name: conv1.conv1.weight False
current name: conv1.conv1.bias False
current name: conv2.conv2.weight False
current name: conv2.conv2.bias False
current name: l1.weight False
current name: l1.bias False
current name: l2.weight False
current name: l2.bias False
current name: l3.weight False
current name: l3.bias False
current name: test.weight False
current name: test.bias False


In [4]:
net = TestNet()
for parameter in net.parameters():
    print(parameter)
    parameter.requires_grad = False#可以对网络参数requires_grad属性进行调整
# for sub_module in net.modules():
#     if isinstance(sub_module, nn.Linear):
#         print('name:', sub_module,'shape:', sub_module.weight.shape, sub_module.weight)
net.add_module('fc', nn.Linear(1,1))

NameError: name 'TestNet' is not defined

In [22]:
for name, parameter in net.state_dict().items():#
    print('name:'name, 'parameter', parameter)#使用这种方式可以按照key读取每一个模块的参数

conv1.conv1.weight tensor([[[[ 0.1985]]],


        [[[ 0.5570]]],


        [[[ 0.8402]]],


        [[[ 0.4672]]],


        [[[-0.0484]]],


        [[[ 0.4602]]],


        [[[ 0.5147]]],


        [[[ 0.1821]]],


        [[[-0.3727]]],


        [[[ 0.4286]]]])
conv1.conv1.bias tensor([ 0.5193, -0.1868, -0.0874,  0.9345,  0.1159,  0.1085, -0.3076,
        -0.6507,  0.1185,  0.9971])
conv2.conv2.weight tensor([[[[-0.4924]]]])
conv2.conv2.bias tensor(1.00000e-02 *
       [ 2.8391])
l1.weight tensor([[-0.1799,  0.2904, -0.2168,  0.0447,  0.2767, -0.3133, -0.2258,
          0.2662,  0.2113,  0.1514],
        [-0.1426, -0.2526,  0.1498, -0.2656,  0.0142,  0.2884, -0.2284,
         -0.0431,  0.2485, -0.2345],
        [-0.1643,  0.1828, -0.2477,  0.0220, -0.1337,  0.0651, -0.3099,
         -0.1271,  0.2799, -0.2499],
        [-0.2457, -0.0152,  0.0334,  0.2132,  0.1115,  0.0639,  0.0008,
         -0.2417, -0.0906,  0.1119],
        [-0.2372,  0.0252,  0.1404, -0.2443,  0.0907, -0.1240, 

注意，在网络模型类中定义的子网络无论是否在forward中调用，都会被保存在对象中，只是说如果使用forward进行反向传播，会对forward指定的网络参数进行优化调整，没有被forward的参数不会发生变化而已。

In [1]:
model = Net()
for name, parameter in model.state_dict().items():
    print(name, parameter.data.shape)#可以看到Conv2d网络的参数shape是(input_channels*output_channels*kernel*kernel)

NameError: name 'Net' is not defined

In [7]:
for i in net.parameters():
    print('shape:',i.size(), type(i))#可以看到对继承nn.Module类来讲，使用parameter()方法获取所有的数据,
for i in net.parameters():
    print('type:',type(i.data), 'tensor data:', i.data)#对parameter.data属性返回的才是tensor类型，之后可以调用tensor类的一些属性和方

shape: torch.Size([10, 1, 1, 1]) <class 'torch.nn.parameter.Parameter'>
shape: torch.Size([10]) <class 'torch.nn.parameter.Parameter'>
shape: torch.Size([1, 1, 1, 1]) <class 'torch.nn.parameter.Parameter'>
shape: torch.Size([1]) <class 'torch.nn.parameter.Parameter'>
shape: torch.Size([10, 10]) <class 'torch.nn.parameter.Parameter'>
shape: torch.Size([10]) <class 'torch.nn.parameter.Parameter'>
shape: torch.Size([15, 10]) <class 'torch.nn.parameter.Parameter'>
shape: torch.Size([15]) <class 'torch.nn.parameter.Parameter'>
shape: torch.Size([1, 15]) <class 'torch.nn.parameter.Parameter'>
shape: torch.Size([1]) <class 'torch.nn.parameter.Parameter'>
shape: torch.Size([15, 15]) <class 'torch.nn.parameter.Parameter'>
shape: torch.Size([15]) <class 'torch.nn.parameter.Parameter'>
shape: torch.Size([1, 1]) <class 'torch.nn.parameter.Parameter'>
shape: torch.Size([1]) <class 'torch.nn.parameter.Parameter'>
type: <class 'torch.Tensor'> tensor data: tensor([[[[ 0.7657]]],


        [[[ 0.8929]]

In [3]:
for i in net.state_dict():#使用字典形式索引
    print(i)
net.state_dict()['fc.weight']#state_dict()中存的是网络的名称和数据参数

NameError: name 'net' is not defined

In [103]:
for i in net.modules():
    print(i)#可以看到在打印的时候会先打印整个网络的架构，然后对自定义网络模块进行解析打印，最后开内部实际的网络结构

TestNet(
  (conv1): SubNet1(
    (conv1): Conv2d(1, 10, kernel_size=(1, 1), stride=(1, 1))
  )
  (conv2): SubNet2(
    (conv2): Conv2d(1, 1, kernel_size=(1, 1), stride=(1, 1))
  )
  (l1): Linear(in_features=10, out_features=10, bias=True)
  (l2): Linear(in_features=10, out_features=15, bias=True)
  (l3): Linear(in_features=15, out_features=1, bias=True)
  (test): Linear(in_features=15, out_features=15, bias=True)
  (fc): Linear(in_features=1, out_features=1, bias=True)
)
SubNet1(
  (conv1): Conv2d(1, 10, kernel_size=(1, 1), stride=(1, 1))
)
Conv2d(1, 10, kernel_size=(1, 1), stride=(1, 1))
SubNet2(
  (conv2): Conv2d(1, 1, kernel_size=(1, 1), stride=(1, 1))
)
Conv2d(1, 1, kernel_size=(1, 1), stride=(1, 1))
Linear(in_features=10, out_features=10, bias=True)
Linear(in_features=10, out_features=15, bias=True)
Linear(in_features=15, out_features=1, bias=True)
Linear(in_features=15, out_features=15, bias=True)
Linear(in_features=1, out_features=1, bias=True)


In [90]:
for i in net.modules():
    if isinstance(i, nn.Conv2d):
        i.weight.data.fill_(0)
        print(type(i))#
#         i.weight.data.copy_()
print(type(net.conv.conv1.weight))#如果使用了子网络嵌套模式，索引名称是一个嵌套形式

<class 'torch.nn.modules.conv.Conv2d'>
<class 'torch.nn.parameter.Parameter'>


>注意：继承`nn.Module`的模块类型是没有weight属性的，但是通过module()返回的每个子模块有weight和bias属性，里面保存了神经网络的参数
nn.Module是没有weight属性的，里面每一个子modlue是有weight属性的，weight和bias是对应参数，使用model.parameters()返回一个可迭代的parameter列表,里面

当访问到tensor级的参数时，就相当方便了，不仅可以使用tensor内置的方法和属性，也可以使用其他的工具包进行处理。

In [49]:
seq = torch.nn.Sequential(*[nn.Linear(1,1) for i in range(3)])
seq.cuda()

Sequential(
  (0): Linear(in_features=1, out_features=1, bias=True)
  (1): Linear(in_features=1, out_features=1, bias=True)
  (2): Linear(in_features=1, out_features=1, bias=True)
)

### 网络输出类型转换tensor转numpy

In [57]:
out = seq(torch.tensor([1.]).cuda())
out.cpu().data.numpy()#首先,不能直接numpy

array([0.27922434], dtype=float32)

不能讲输出的tensor直接转换为numpy,需要使用.data进行调用,相当于把纯数字的部分(不包含梯度)取出来才可以
* 此外,如果是cuda类型的数据,首先是需要转换为cpu()才可以正常调用

In [125]:
class TestNet(nn.Module):
    def __init__(self):
        super(TestNet, self).__init__()
        self.fc1 = nn.Linear(10,20)
        self.dropLayer = nn.Dropout2d(0.5)
        self.fc2 = nn.Linear(20,1)
    def forward(self, x):
        x = self.fc1(x)
        x = self.dropLayer(x)
        return self.fc2(x)

model = TestNet()
model

TestNet(
  (fc1): Linear(in_features=10, out_features=20, bias=True)
  (dropLayer): Dropout2d(p=0.5)
  (fc2): Linear(in_features=20, out_features=1, bias=True)
)

In [81]:
x = torch.rand(10).unsqueeze(0)#送入网络的维度要高一维
x

tensor([[ 0.4621,  0.8525,  0.0080,  0.4500,  0.0087,  0.2150,  0.7558,
          0.7200,  0.5114,  0.3416]])

In [254]:
model.train()

TestNet(
  (fc1): Linear(in_features=10, out_features=20, bias=True)
  (dropLayer): Dropout2d(p=0.5)
  (fc2): Linear(in_features=20, out_features=1, bias=True)
)

In [296]:
model.eval()#通过设置eval,dropout的随机性会消失,再次设置为train打开
model(x)

tensor([[-0.1677]])

In [ ]:
regex = r".*/\w+(\d{4})(\d{2})(\d{2})T(\d{2})(\d{2})/mask\_rcnn\_\w+(\d{4})\.pth"#########正则表达式
m = re.match(regex, model_path)
if m:
    now = datetime.datetime(int(m.group(1)), int(m.group(2)), int(m.group(3)),
                            int(m.group(4)), int(m.group(5)))
    self.epoch = int(m.group(6))

# Directory for training logs
self.log_dir = os.path.join(self.model_dir, "{}{:%Y%m%dT%H%M}".format(
self.config.NAME.lower(), now))

# Path to save after each epoch. Include placeholders that get filled by Keras.
self.checkpoint_path = os.path.join(self.log_dir, "mask_rcnn_{}_*epoch*.pth".format(
self.config.NAME.lower()))
self.checkpoint_path = self.checkpoint_path.replace(
"*epoch*", "{:04d}")